### Goal: Are soccer referees more likely to give red cards to dark-skin-toned players than light-skin-tonned players?

- Before plotting/joining/doing something, have a question or hypothesis that you want to investigate (we know what to determine here)
- Draw a plot of what you want to see on paper to sketch the idea
- Write it down, then make the plan on how to get there
- How do you know that you're not fooling yourself
- What else can I check if this is actually true?
- What evidence could there be that it's wrong?

### It is the red cards obtained in the entire career
- When ref A was refering a game that involved player B, then that had a line on the dataset. If ref A and player B were involved in multiple games, it still had only 1 line in the dataset

In [1]:
%matplotlib inline
%config InlineBackend.figure_forest='retina'

IPython has a set of predefined ‘magic functions’ that you can call 
with a command line style syntax. There are two kinds of magics, 
line-oriented and cell-oriented. Line magics are prefixed with 
the % character and work much like OS command-line calls: they get 
as an argument the rest of the line, where arguments are passed without 
parentheses or quotes. Lines magics can return results and can be used 
in the right hand side of an assignment. Cell magics are prefixed with 
a double %%, and they are functions that get as an argument not only the 
rest of the line, but also the lines below it in a separate argument.

In [2]:
from __future__ import absolute_import, division, print_function
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib.pyplot import GridSpec
import seaborn as sns
#import mpld3
import numpy as np
import pandas as pd
import os, sys
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
sns.set_context("poster", font_scale=1.3)

#import missingno as msno
#import pandas_profiling

#import hdbscan
from sklearn.datasets import make_blobs
import time

In [3]:
df = pd.read_csv("redcard.csv.gz", compression='gzip')

In [4]:
df.head()

,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,victories,...,rater2,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
0,lucas-wilchez,Lucas Wilchez,Real Zaragoza,Spain,31.08.1983,177.0,72.0,Attacking Midfielder,1,0,...,0.50,1,1,GRC,0.326391,712.0,0.000564,0.396000,750.0,0.002696
1,john-utaka,John Utaka,Montpellier HSC,France,08.01.1982,179.0,82.0,Right Winger,1,0,...,0.75,2,2,ZMB,0.203375,40.0,0.010875,-0.204082,49.0,0.061504
2,abdon-prats,Abdón Prats,RCD Mallorca,Spain,17.12.1992,181.0,79.0,NaN,1,0,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
3,pablo-mari,Pablo Marí,RCD Mallorca,Spain,31.08.1993,191.0,87.0,Center Back,1,1,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
4,ruben-pena,Rubén Peña,Real Valladolid,Spain,18.07.1991,172.0,70.0,Right Midfielder,1,1,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002


In [5]:
df.shape

(146028, 28)

In [6]:
df.describe().T # no difference wrt df.describe()

,count,mean,std,min,25%,50%,75%,max
height,145765.0,181.935938,6.738726,1.610000e+02,177.000000,182.000000,187.000000,2.030000e+02
weight,143785.0,76.075662,7.140906,5.400000e+01,71.000000,76.000000,81.000000,1.000000e+02
games,146028.0,2.921166,3.413633,1.000000e+00,1.000000,2.000000,3.000000,4.700000e+01
victories,146028.0,1.278344,1.790725,0.000000e+00,0.000000,1.000000,2.000000,2.900000e+01
ties,146028.0,0.708241,1.116793,0.000000e+00,0.000000,0.000000,1.000000,1.400000e+01
defeats,146028.0,0.934581,1.383059,0.000000e+00,0.000000,1.000000,1.000000,1.800000e+01
goals,146028.0,0.338058,0.906481,0.000000e+00,0.000000,0.000000,0.000000,2.300000e+01
yellowCards,146028.0,0.385364,0.795333,0.000000e+00,0.000000,0.000000,1.000000,1.400000e+01
yellowReds,146028.0,0.011381,0.107931,0.000000e+00,0.000000,0.000000,0.000000,3.000000e+00
redCards,146028.0,0.012559,0.112889,0.000000e+00,0.000000,0.000000,0.000000,2.000000e+00


In [7]:
df.dtypes

playerShort       object
player            object
club              object
leagueCountry     object
birthday          object
height           float64
weight           float64
position          object
games              int64
victories          int64
ties               int64
defeats            int64
goals              int64
yellowCards        int64
yellowReds         int64
redCards           int64
photoID           object
rater1           float64
rater2           float64
refNum             int64
refCountry         int64
Alpha_3           object
meanIAT          float64
nIAT             float64
seIAT            float64
meanExp          float64
nExp             float64
seExp            float64
dtype: object

In [8]:
all_columns = df.columns.tolist()
all_columns

['playerShort',
 'player',
 'club',
 'leagueCountry',
 'birthday',
 'height',
 'weight',
 'position',
 'games',
 'victories',
 'ties',
 'defeats',
 'goals',
 'yellowCards',
 'yellowReds',
 'redCards',
 'photoID',
 'rater1',
 'rater2',
 'refNum',
 'refCountry',
 'Alpha_3',
 'meanIAT',
 'nIAT',
 'seIAT',
 'meanExp',
 'nExp',
 'seExp']

In [9]:
type(all_columns)

list

### Potential Issue: Ratings were given by 2 people

- How do I combine rater1 and rater2? Average them? What if they disagree? Throw it out?
- Is data imbalanced i.e. red cards are very rare?
- Is data biased i.e. players have different amounts of playtime? Is this the summary of their whole career?
- How do I know I have accounted for all types of confounding?

### First, is there systematic discrimination across all refs?

Exploration/Hypothesis

- Distribution of games played
- Red cards vs games played
- Reds per game played vs total cards per game played by skin colur
- Distribution of #red, #yellow, total cards and fraction red per game played for all players by abg. skin colour
- How many refs did players encounter?
- Did some clubs play more aggressively and get carded more? Or are more reserved and got less?
- Does carding vary by leagueCountry?
- Do high scorers get more slack (fewer cards) for the same position?
- Are there some referees who give more yellow/red cards than others?
- How consistent are raters? Check with Cohen's kappa
- How do red cards vary by position? Ex. defenders get more?
- Do players with more games get more cards, and is there difference across skin colour?
- Indication of bias on refCountry?

In [10]:
df['height'].mean() # same as df.height.mean()

# takes avg. of all the players, even if the player is listed (say, 10 times for 10 different refs). This affects the avg.

181.93593798236887

In [11]:
np.mean(df.groupby('playerShort').height.mean())

# groups the players, and counts the player only once to calculate the avg.

181.74372848007872

### To break the dataset into different observational units. Thus, separate tables (or dataframes) for:
- players
- clubs
- referees
- countries
- dyads (something that consists of two elements or parts. Ex. "the mother–child dyad")

### Create Tidy Players Table

In [12]:
# This table will have a unique ID per player

player_index = 'playerShort'
player_cols = ['birthday','height','weight','position','photoID','rater1','rater2']

In [13]:
# count the unique variables (if we got different weight value, for example, then we should get more than 1 unique value
# in this groupby clause)

all_cols_unique_players = df.groupby('playerShort').agg({col:'nunique' for col in player_cols})

In [14]:
all_cols_unique_players.head()

,birthday,height,weight,position,photoID,rater1,rater2
playerShort,,,,,,,
aaron-hughes,1,1,1,1,1,1,1
aaron-hunt,1,1,1,1,1,1,1
aaron-lennon,1,1,1,1,1,1,1
aaron-ramsey,1,1,1,1,1,1,1
abdelhamid-el-kaoutari,1,1,1,1,1,1,1


 #### The above code can be understood as:
 
 If player A had the birthday mentioned as xx/xx/xxxx from ref1 and a different birthday yy/yy/yyyy from ref2, the birthday columnabove would have the value 2

In [15]:
all_cols_unique_players.shape

(2053, 7)

In [16]:
# If all values are the same for players, then this should be empty (and it is !)

all_cols_unique_players[all_cols_unique_players>1].dropna().head()

,birthday,height,weight,position,photoID,rater1,rater2
playerShort,,,,,,,


In [17]:
# A slightly more elegant way to test the uniqueness

all_cols_unique_players[all_cols_unique_players>1].dropna().shape[0] == 0

True

#### To create a function which performs the same operations as above. It will be used for all the other Tidy Tables created

In [18]:
def get_subgroup(dataframe, g_index, g_columns):
    """Helper function that creates a sub-table from the columns and runs a quick uniqueness test"""
    g = dataframe.groupby(g_index).agg({col:'nunique' for col in g_columns})
    if g[g>1].dropna().shape[0] != 0:
        print("Warning: you assumed this had all unique values but it doesn't.")
    return dataframe.groupby(g_index).agg({col:'max' for col in g_columns})

In [19]:
players = get_subgroup(df, player_index, player_cols)
players.head()

,birthday,height,weight,position,photoID,rater1,rater2
playerShort,,,,,,,
aaron-hughes,08.11.1979,182.0,71.0,Center Back,3868.jpg,0.25,0.00
aaron-hunt,04.09.1986,183.0,73.0,Attacking Midfielder,20136.jpg,0.00,0.25
aaron-lennon,16.04.1987,165.0,63.0,Right Midfielder,13515.jpg,0.25,0.25
aaron-ramsey,26.12.1990,178.0,76.0,Center Midfielder,94953.jpg,0.00,0.00
abdelhamid-el-kaoutari,17.03.1990,180.0,73.0,Center Back,124913.jpg,0.25,0.25


In [20]:
# Save this for future use.

def save_subgroup(dataframe, g_index, subgroup_name, prefix='../RedCard/'):
    save_subgroup_filename = "".join([prefix, subgroup_name, ".csv.gz"])
    dataframe.to_csv(save_subgroup_filename, compression='gzip')
    test_df = pd.read_csv(save_subgroup_filename, compression='gzip', index_col=g_index)
    
    # Test that we recover what we send in
    
    if dataframe.equals(test_df):
        print("Test passed: we recovered the equivalent subgroup dataframe")
    else:
        print("Warning -- equivalence test !!! Double-check.")

In [21]:
save_subgroup(players,player_index,'players')

Test passed: we recovered the equivalent subgroup dataframe


### Create Tidy Clubs Table

In [22]:
club_index = 'club'
club_cols = ['leagueCountry']
clubs = get_subgroup(df, club_index, club_cols)
clubs.head()

,leagueCountry
club,
1. FC Nürnberg,Germany
1. FSV Mainz 05,Germany
1899 Hoffenheim,Germany
AC Ajaccio,France
AFC Bournemouth,England


In [23]:
clubs.shape

(118, 1)

In [24]:
clubs['leagueCountry'].value_counts()

England    48
Spain      27
France     22
Germany    21
Name: leagueCountry, dtype: int64

In [25]:
save_subgroup(clubs, club_index, "clubs")

Test passed: we recovered the equivalent subgroup dataframe


### Create Tidy Referees Table

In [26]:
referee_index = 'refNum'
referee_cols = ['refCountry']
referees = get_subgroup(df, referee_index, referee_cols)
referees.head()

,refCountry
refNum,
1,1
2,2
3,3
4,4
5,5


In [27]:
referees.refCountry.nunique()

161

In [28]:
referees.shape

(3147, 1)

In [29]:
save_subgroup(referees, referee_index, "referees")

Test passed: we recovered the equivalent subgroup dataframe


### Create Tidy Countries Table

In [30]:
country_index = 'refCountry'
country_cols = ['Alpha_3', # rename this as name of country
                'meanIAT',
                'nIAT',
                'seIAT',
                'meanExp',
                'nExp',
                'seExp',
                ]
countries = get_subgroup(df, country_index, country_cols)
countries.head()

# IAT = Implicit Association Test, for countries how implicitly biased they are

,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
refCountry,,,,,,,
1,GRC,0.326391,712.0,0.000564,0.396000,750.0,0.002696
2,ZMB,0.203375,40.0,0.010875,-0.204082,49.0,0.061504
3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
5,TUN,0.167132,19.0,0.027327,-0.789474,19.0,0.111757


In [31]:
rename_columns = {'Alpha_3':'countryname'}
countries = countries.rename(columns=rename_columns)
countries.head()

,countryname,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
refCountry,,,,,,,
1,GRC,0.326391,712.0,0.000564,0.396000,750.0,0.002696
2,ZMB,0.203375,40.0,0.010875,-0.204082,49.0,0.061504
3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
5,TUN,0.167132,19.0,0.027327,-0.789474,19.0,0.111757


In [32]:
countries.shape

(161, 7)

In [33]:
save_subgroup(countries, country_index, "countries")

Warning -- equivalence test !!! Double-check.


#### The warning is due to a precision error of 10^-9. 

### Now, to create a separate (not yet Tidy) Dyads Table

In [34]:
dyad_index = ['refNum', 'playerShort']
dyad_columns = ['games',
               'victories',
               'ties',
               'defeats',
               'goals',
               'yellowCards',
               'yellowReds',
               'redCards',
               ]

In [35]:
dyads = get_subgroup(df, dyad_index, dyad_columns)

In [36]:
dyads.head(10) # it lists out the referees shared among multiple players

games  victories  ties  defeats  goals  \
refNum playerShort                                                  
1      lucas-wilchez           1          0     0        1      0   
2      john-utaka              1          0     0        1      0   
3      abdon-prats             1          0     1        0      0   
       pablo-mari              1          1     0        0      0   
       ruben-pena              1          1     0        0      0   
4      aaron-hughes            1          0     0        1      0   
       aleksandar-kolarov      1          1     0        0      0   
       alexander-tettey        1          0     0        1      0   
       anders-lindegaard       1          0     1        0      0   
       andreas-beck            1          1     0        0      0   

                           yellowCards  yellowReds  redCards  
refNum playerShort                                            
1      lucas-wilchez                 0           0         0  
2      john-utaka                    1           0         0  
3      abdon-prats                   1           0         0  
       pablo-mari                    0           0         0  
       ruben-pena                    0           0         0  
4      aaron-hughes                  0           0         0  
       aleksandar-kolarov            0           0         0  
       alexander-tettey              0           0         0  
       anders-lindegaard             0           0         0  
       andreas-beck                  0           0         0

In [37]:
dyads.shape

(146028, 8)

In [38]:
dyads[dyads.redCards > 1].head(10)

games  victories  ties  defeats  goals  yellowCards  \
refNum playerShort                                                              
140    bodipo                 6          2     1        3      1            0   
367    antonio-lopez_2        8          5     2        1      0            2   
432    javi-martinez         14          4     3        7      2            2   
       jonas                  9          1     4        4      1            0   
487    phil-jagielka          7          2     1        4      1            0   
586    cyril-jeunechamp      14          8     0        6      0            6   
804    sergio-ramos          18         12     1        5      4            6   
985    aly-cissokho           9          1     5        3      1            1   
1114   eugen-polanski         8          4     0        4      0            0   
1214   emmanuel-adebayor     23          9     7        7     10            4   

                          yellowReds  redCards  
refNum playerShort                              
140    bodipo                      0         2  
367    antonio-lopez_2             0         2  
432    javi-martinez               0         2  
       jonas                       0         2  
487    phil-jagielka               0         2  
586    cyril-jeunechamp            0         2  
804    sergio-ramos                1         2  
985    aly-cissokho                0         2  
1114   eugen-polanski              0         2  
1214   emmanuel-adebayor           1         2

In [39]:
save_subgroup(dyads, dyad_index, "dyads")

Test passed: we recovered the equivalent subgroup dataframe
